In [1]:
#libraries
import json
from collections import defaultdict
import time
import csv
import os

In [16]:
#import data from folder
base_path = "/nas/home/slnagark/Genome/datasets/"

def get_scene_graphs():
    with open(os.path.join(base_path, 'scene_graphs.json')) as f:
        scene_graphs = json.load(f)
    return scene_graphs

def get_image_data():
    with open(os.path.join(base_path, 'image_data.json')) as f:
        image_data = json.load(f)
    return image_data

def get_region_descriptions():
    with open(os.path.join(base_path, 'region_descriptions.json')) as f:
        region_descriptions = json.load(f)
    return region_descriptions

def get_objects():
    with open(os.path.join(base_path, 'objects.json')) as f:
        objects = json.load(f)
    return objects

def get_relationships():
    with open(os.path.join(base_path, 'relationships.json')) as f:
        relationships = json.load(f)
    return relationships

def get_region_graphs():
    with open(os.path.join(base_path, 'region_graphs.json')) as f:
        region_graphs = json.load(f)
    return region_graphs

def get_relationships():
    with open(os.path.join(base_path, 'relationships.json')) as f:
        relationships = json.load(f)
    return relationships

def get_relationship_synsets():
    with open(os.path.join(base_path, 'relationship_synsets.json')) as f:
        relationship_synets = json.load(f)
    return relationship_synets

def get_attribute_synsets():
    with open(os.path.join(base_path, 'attribute_synsets.json')) as f:
        attribute_synets = json.load(f)
    return attribute_synets

def get_attributes():
    with open(os.path.join(base_path, 'attributes.json')) as f:
        attributes = json.load(f)
    return attributes

def get_object_synsets():
    with open('/nas/home/slnagark/Genome/datasets/object_synsets.json') as f:
        object_synets = json.load(f)
    return object_synets

def get_question_answers():
    with open(os.path.join(base_path, 'question_answers.json')) as f:
        q_a = json.load(f)
    return q_a

def get_mapping_qa_region():
    with open(os.path.join(base_path,'qa_to_region_mapping.json')) as f:
        map_ = json.load(f)
    return map_

In [17]:
obj = get_objects()

In [40]:

img2obj = {}

for ind, images in enumerate(obj):
    image_id = images["image_id"]
    objid2info = {}
    for ob in images["objects"]:
        cnt+=1
        ob_id = ob["object_id"]
        ob_name = ob["names"]
        ob_syns = ob["synsets"]
        objid2info[ob_id]={
            "merged":ob['merged_object_ids'],
            "names":ob_name,
            "synset":ob_syns
        }
    img2obj[image_id] = objid2info
    if ind==9:
        break


In [3]:
start = time.time()
attr_ = get_attributes()
print("Done importing Attributes in : ",time.time()-start," sec.")

Done importing Attributes in :  22.312848806381226  sec.


In [4]:
start = time.time()
map_ = get_mapping_qa_region()
print("Done importing Mapping QA to Regions in : ",time.time()-start," sec.")

Done importing Mapping QA to Regions in :  0.6207714080810547  sec.


In [5]:
start = time.time()
q_a_ = get_question_answers()
print("Done importing question and answers  in : ",time.time()-start," sec.")

Done importing question and answers  in :  6.247305154800415  sec.


In [6]:
start = time.time()
scene_graphs_ = get_scene_graphs()
print("Done importing scene_graphs in : ",time.time()-start," sec.")

Done importing scene_graphs in :  26.920303344726562  sec.


In [7]:
start = time.time()
region_graphs_ = get_region_graphs()
print("Done importing region graphs in : ",time.time()-start," sec.")

Done importing region graphs in :  96.97917461395264  sec.


In [8]:
start = time.time()
relationships_ = get_relationships()
print("Done importing relationships in : ",time.time()-start," sec.")

Done importing relationships in :  12.799816608428955  sec.


In [9]:
#Initialize region id to question id mappings:
r_id2qid = defaultdict(list)
for q_id, reg_id in map_.items():
    r_id2qid[int(reg_id)].append(int(q_id))


In [115]:
"""
input: relationship.json

output: list of dictionaries of relationships
        [
            {
                "rel_id" : int,
                "rel_name" : str,
                "rel_syns" : [str],
                "sub_id" : int,
                "sub_name" : str,
                "sub_syns" : [str],
                "ob_id" : int,
                "ob_name" : str,
                "ob_syns" : [str]
            },
            ....
        ]
"""
relations_image_to_relations = {}
relationid2info = {}
def get_info_from_relations(relationships):

    for ind, relations in enumerate(relationships):
        relations_id = []
 
        for rels in relations["relationships"]:
            rel_id = int(rels["relationship_id"])
            ob_ids = []
            sub_ids = []
            o_name = "name"
            s_name = "name"
            ob_ids.append(int(rels["object"]["object_id"]))
            sub_ids.append(int(rels["subject"]["object_id"]))
            if "merged_object_ids" in rels["object"].keys():
                o_name = "names"
                ob_ids += list(set(rels["object"]["merged_object_ids"]))
            if len(rels["subject"].keys())!=7:  #if it has merged object ids
                s_name = "names"
                sub_ids += list(set(rels["subject"]["merged_object_ids"]))
            info = {
                        "rel_id" : rel_id,
                        "rel_name" : rels["predicate"].lower(),
                        "rel_syns" : rels["synsets"],
                        "sub_id" : sub_ids,
                        "sub_name" : rels["subject"][s_name],
                        "sub_syns" : rels["subject"]["synsets"],
                        "ob_id" : ob_ids,
                        "ob_name" : rels["object"][o_name],
                        "ob_syns" : rels["object"]["synsets"]
                  }
            relations_id.append(info)
            relationid2info[rel_id] = info
            
        relations_image_to_relations[relations["image_id"]] = relations_id
        if ind == 9:
                break

In [116]:
get_info_from_relations(relationships_)

In [143]:
"""
input: scene_graphs.json

output: dictionary of list of dictionaries of relationships
        {
          image_id:  
          [
              {
                "rel_id" : int,
                "rel_name" : str,
                "rel_syns" : [str],
                "sub_id" : int,
                "sub_name" : str,
                "sub_syns" : [str],
                "ob_id" : int,
                "ob_name" : str,
                "ob_syns" : [str]
              },
              ....
          ]
        }
"""
#scene_graphs_image_to_relations_temp = {}
scene_graphs_image_to_relations = {}
relationid2name = defaultdict(list)
relationid2syns = defaultdict(list)

def get_info_from_scene_graphs(scene_graphs):
    for ind, an_image in enumerate(scene_graphs):
        image_id = an_image['image_id']
        relationid2infos = {}
        # OBJECTS
        objid2names=defaultdict(list)
        objid2syns=defaultdict(list)
        for o in an_image['objects']:
            obj_id = int(o['object_id'])
            objid2syns[obj_id] = o['synsets']
            for name in o['names']:
                objid2names[obj_id].append(name)
        
        # RELATIONS
        relations_id = []
        for rel in an_image['relationships']:
            rel_id = int(rel['relationship_id'])
            
            if rel_id not in relationid2name.keys():
            #if rel['predicate'].lower() not in relationid2name[rel_id]:
                relationid2name[rel_id].append(rel['predicate'].lower())
                relationid2syns[rel_id].append(rel["synsets"])
                
            sub_id= int(rel['subject_id'])
            obj_id= int(rel['object_id'])
            info = {
                        "rel_id" : rel_id,
                        "rel_name" : relationid2name[rel_id],
                        "rel_syns" : relationid2syns[rel_id],
                        "sub_id" : sub_id,
                        "sub_name" : objid2names[sub_id],
                        "sub_syns" : objid2syns[sub_id],
                        "ob_id" : obj_id,
                        "ob_name" : objid2names[obj_id],
                        "ob_syns" :objid2syns[obj_id]
                    }
            
            #relations_id.append(info)
            relationid2infos[rel_id] = info
        #scene_graphs_image_to_relations_temp[image_id] = relations_id
        scene_graphs_image_to_relations[image_id] = relationid2infos#relations_id
        if ind == 9:
            break
            
get_info_from_scene_graphs(scene_graphs_)

In [136]:
"""
input: region_graphs.json

output: dictionary of list of dictionaries of regions
        {
          image_id:  
          [
              {
                "region_id" : int,
                "phrase" : str,
                "relationship_id" : int,
              },
              ....
          ]
        }
"""
rgraphs_image_to_relation = {}

def get_info_from_region_graphs(region_graphs):
    
    
    for ind, reg_graphs in enumerate(region_graphs):
        region_data = []
        region_info = []
        image_id = int(reg_graphs["image_id"])
        
        
        for regions in reg_graphs['regions']:
            rel = []
            reg_info = {}
            
            r_objid2name = defaultdict(list)
            r_objid2syns = defaultdict(list)
                
            region_id = int(regions["region_id"])
            phrase = regions["phrase"]      
            reg_info["region_id"] = region_id
            reg_info["phrase"] = phrase
            
            #relationship keys missing.
            for obj in regions["objects"]:
                ob_id = int(obj["object_id"])
                r_objid2name[ob_id].append(obj["name"])
                r_objid2syns[ob_id].append(obj["synsets"])
     
            for relations in regions["relationships"]:
                r_id = int(relations["relationship_id"])
            
                if r_id not in relationid2name.keys():
                    #most of the time won't execute since all the relations are present.
                    relationid2name[r_id].append(relations["predicate"].lower())
                    relationid2syns[r_id].append(relations["synsets"])
                    region_info.append({
                        "rel_id" : relations["relationship_id"],
                        "rel_name" : relations["predicate"],
                        "rel_syns" : relations["synsets"],
                        "sub_id" : relations["subject_id"],
                        "obj_id" : relations["object_id"],
                        "sub_name" : r_objid2name[relations["subject_id"]],
                        "obj_name" : r_objid2name[relations["object_id"]],
                        "sub_syns" : r_objid2syns[relations["subject_id"]],
                        "obj_syns" : r_objid2syns[relations["object_id"]]
                    })
                
                rel.append(r_id)
            
            reg_info["relationship_id"] = rel    
            region_data.append(reg_info)
        
        if len(region_info)!=0:
            print("New Relations found other than ones in SceneGraphs")
            scene_graphs_image_to_relations[image_id].extend(region_info)
            #Mostly this wont execute since all the relations are present in scene_graphs
            
        rgraphs_image_to_relation[image_id] = region_data
        
        if ind == 9:
            break

In [137]:
get_info_from_region_graphs(region_graphs_)


In [138]:
"""
input: question_answers.json

output: dictionary of dictionary
        {
            "image_id" : {
                            "loc" : [{
                                        "question": str
                                        "answer"  : str
                                    }],
                                    
                            "time" : [{
                                        "question": str
                                        "answer"  : str
                                     }],
                                     
                            "event" :[{
                                        "question": str
                                        "answer"  : str
                                     }],
                          }
        }
"""
image_to_qa = {}
q_id2params = {}
q_id2q = {}
def get_info_from_q_a(q_a):

    for ind, qa in enumerate(q_a):
        loc = []
        time = []
        event = []
        cnt = 0
        image_id = qa["id"]
        for qas in qa["qas"]:
            qa_id = int(qas["qa_id"])
            question = qas["question"]
            q_id2q[qa_id] = question
            answer = qas["answer"]
            q = question.split(" ")
            
            
            if q[0].lower() == "where":
                q_id2params[qa_id] = {
                    "type" : "location",
                    "question" : question,
                    "answer" : answer
                }
                loc.append({
                    "question" : question,
                    "answer" : answer
                })
                
            elif q[0].lower() =="when":
                q_id2params[qa_id] = {
                    "type" : "time",
                    "question" : question,
                    "answer" : answer
                } 
                time.append({"question" : question, "answer" : answer})
              
            elif "doing" in question:
                q_id2params[qa_id] = {
                    "type" : "event",
                    "question" : question,
                    "answer" : answer
                }
                event.append({
                    "question" : question,
                    "answer" : answer
                })
    
        
        image_to_qa[image_id] = {
            "location" : loc,
            "time" : time,
            "event" : event
        }
        if ind ==9:
            break


In [139]:
get_info_from_q_a(q_a_)

In [140]:
"""
input: Input from attributes.json

output: {
            "image_id" : {
                            "object_id": attributes [str]
                          }
        }

"""
imgid2ob_att = {}
def get_info_from_attr(attr):
    
    for ind, img in enumerate(attr):
        img_id = img["image_id"]
        obid2attr = defaultdict(list)
        for attrs in img["attributes"]:
            ob_id = attrs["object_id"]
            if "attributes" not in attrs.keys():
                attributes = []
            else:    
                attributes = attrs["attributes"]
            obid2attr[ob_id].append(attributes)

        imgid2ob_att[img_id] = obid2attr
        if ind ==9:
            break


In [141]:
get_info_from_attr(attr_)

In [132]:
#Sample Output for image 1

path = "/nas/home/slnagark/Genome/"

cnt = 0
i=0
j=0
with open(os.path.join(path,'output2.tsv'), 'wt') as out_file:
    tsv_writer = csv.writer(out_file, delimiter='\t')
    tsv_writer.writerow(['image_id', 'region_id', 'relationship', 'subject', 
                         'object','subject_synset','object_synset', 'relationship_synset', 'sentence', 
                        'location', 'time', 'event'])
    
    for ind in range(10):
        for reg in rgraphs_image_to_relation[ind+1]:
            reg_id = reg["region_id"]  #get region id
            loc = []
            time = []
            event = []
            for q_ids in r_id2qid[reg_id]:  #get questions from this region

                if q_ids in q_id2params.keys():
                    cnt+=1
                    if q_id2params[q_ids]["type"] == "location":
                        loc.append(q_id2params[q_ids]["question"])
                    elif q_id2params[q_ids]["type"] == "time":
                        time.append(q_id2params[q_ids]["question"])
                    elif q_id2params[q_ids]["type"] == "event":
                        event.append(q_id2params[q_ids]["question"])   

            if reg["relationship_id"] != []: #if relationship exists then only output into tsv
                
                for rel_id in reg["relationship_id"]:
#                     rel = relationid2info[rel_id]
                    rel = scene_graphs_image_to_relations[ind+1][rel_id]
                    output = {
                        "r_id" : reg["region_id"],
                        "relationship" : rel['rel_name'],
                        "subject" : rel["sub_name"],
                        "object" : rel["ob_name"],
                        "subject_synset" : rel["sub_syns"],
                        "object_synset" : rel["ob_syns"],
                        "r_synset" : rel["rel_syns"],
                        "phrase" : reg["phrase"],
                    }


                    tsv_writer.writerow([
                        ind+1, output["r_id"],output["relationship"],output["subject"],output["object"],
                        output["subject_synset"], output["object_synset"],output["r_synset"], output["phrase"],
                        loc, time, event
                    ])
